In [1]:
# 추천을 하고자 하는 컨텐츠기반 룰을 정하고,
# 물건을 3개 이상 추천하는 추천시스템을 완성하시오.
# db의 데이터를 임의로 추가할 수 있음.
# 함수나 클래스로 모듈화해주세요.

In [2]:
import pymysql
import pandas as pd
from collections import Counter

In [3]:
# 1. board의 제목이나 내용에서 가장 많은 제품

# db 연동 함수
def connect():
    con = pymysql.connect(
        host = 'localhost', 
        port = 3306, # int값
        user = 'root', 
        password = 'woody1234', 
        db = 'mldb3', 
        charset = 'utf8'
    )
    
#     print("1. 연결성공")

    cur = con.cursor()
    
#     print("2. 커서 객체 획득")
    
    return cur, con

In [5]:
def sql_select1(cur, customer):
    sql = """
    select p.id , p.name , b.title , b.writer
    from product p
    inner join board b
    on p.name = b.title and b.writer != %s
    order by p.id
    """
    result = cur.execute(sql, customer)
    return result

In [6]:
def out_data(cur):
    return cur.fetchall()

In [9]:
# 1. db 연결
cur, con = connect()

# 2. 고객 정보
customer = input("customer's name: ")

# 3. sql execute
sql_select1(cur, customer)

# 4. data
data = out_data(cur)

# 5. dataframe => 상위 3개 추출
df = pd.DataFrame(data, columns=['product_id', 'product_name', 'board_title', 'board_writer'])
counter_product = Counter(df['product_name'])
recom = counter_product.most_common(3)

# 6. 추천
print(f"\nWelcome {customer}!! {customer}님께 추천해드릴 제품은...\n")
print(f"1. {recom[0][0]}")
print(f"2. {recom[1][0]}")
print(f"3. {recom[2][0]}")
print("\n입니다!!")


customer's name: fuck

Welcome fuck!! fuck님께 추천해드릴 제품은...

1. jsp
2. java
3. spring

입니다!!


In [19]:
def sql_select2(cur2, product):
    sql="""
    select p.id , p.price
    from product p
    where p.name = %s
    """
    result = cur2.execute(sql, product)
    return result

In [20]:
def sql_insert(con, cur, data):
    sql = """
    insert into orderlist values (null, %s, %s, %s)
    """
    result2 = cur.execute(sql, data)
    con.commit()
    return result2

In [21]:
# 7. 구매
order_question = input("구매하시겠습니까?(yes/no) ")

if order_question == "yes":
    
    print("-------------구매를 진행합니다.-------------")
    your_id = input("당신의 id는 ")
    order = input("어떤 것을 구매하시겠습니까? ")
    
    # 구매하고자 하는 제품의 id와 price
    sql_select2(cur, order) 
    product_info = out_data(cur)
    
    # orderlist에 추가할 정보
    data_list = [your_id, product_info[0][0], int(product_info[0][1])]
    result3 = sql_insert(con, cur, data_list)
    
    if result3 == 1:
        print("----구매가 완료되었습니다.")
    else:
        print("----구매에 문제가 발생했습니다.") 
        
elif order_question == "no":
    print("-------------아쉽지만 다음을 기대할게요!!-------------")

else:
    print("답변 오류 (yes/no로 입력해주세요.)")

구매하시겠습니까?(yes/no) yes
-------------구매를 진행합니다.-------------
당신의 id는 fuck
어떤 것을 구매하시겠습니까? jsp
(('101', '2000'),)
----구매가 완료되었습니다.
